In [161]:
# Load libraries
import numpy as np
from numpy import linalg
from scipy.special import digamma
from scipy.special import polygamma

In [162]:
# Set seed
np.random.seed(10)

In [163]:
# Doc params
V = 5 # Number of unique possible words
N = 10 # Number of words PER DOCUMENT (here all docs have same number of words)
K = 2 # Number of topics
M = 10 # Number of documents

In [164]:
# Set corpus parameters
alpha_true = np.array([5, 3]) # Parameters for sampling topic distribution
beta_true = np.zeros((K, V)) # Parameters for sampling word distribution per topic
beta_true[0, :] = np.array([0.7, 0.2, 0.1, 0, 0])
beta_true[1, :] = np.array([0, 0, 0.3, 0.3, 0.4])

In [165]:
# Simulate theta, topic distributions for each doc
theta = np.random.dirichlet(alpha_true, size = M)
theta

array([[ 0.67070431,  0.32929569],
       [ 0.78771623,  0.21228377],
       [ 0.64892707,  0.35107293],
       [ 0.52370171,  0.47629829],
       [ 0.59934066,  0.40065934],
       [ 0.87273546,  0.12726454],
       [ 0.37528339,  0.62471661],
       [ 0.46307675,  0.53692325],
       [ 0.68783733,  0.31216267],
       [ 0.7164995 ,  0.2835005 ]])

In [166]:
# Simulate data for one document
z = np.zeros((M, N), dtype = int) # Sample topics
for i in range(M):
    for j in range(N):
        z[i, j] = np.nonzero(np.random.multinomial(n = 1, pvals = theta[i,]))[0]
z

array([[0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 1]])

In [167]:
w = np.zeros((M, N), dtype = int) # Sample words
for i in range(M):
    for j in range(N):
        w[i, j] = np.nonzero(np.random.multinomial(n = 1, pvals = beta_true[z[i, j], :]))[0]
w

array([[0, 2, 0, 4, 0, 3, 0, 0, 0, 0],
       [4, 1, 0, 3, 2, 0, 4, 0, 0, 3],
       [1, 0, 0, 0, 0, 1, 0, 3, 0, 0],
       [2, 3, 1, 0, 2, 4, 4, 1, 4, 1],
       [1, 2, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 3, 0, 0, 3, 0, 0, 0, 2, 0],
       [4, 2, 2, 1, 2, 4, 2, 2, 4, 3],
       [1, 3, 3, 4, 3, 2, 0, 3, 4, 0],
       [0, 3, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 3, 2, 0, 1, 0, 2, 0, 2]])

In [168]:
B = np.zeros((N, V, M)) # Wordbank, where each row is a 1-hot vector
for i in range(M):
    for n in range(N):
        B[n, w[i, n], i] = 1

In [169]:
# E-step, update phi and gamma
def E_step(alpha, beta, w, N, K, tol = 1e-6):
    phi_old = np.ones((N, K))
    phi_old /= K
    phi_new = np.ones((N, K))
    gamma_old = alpha + N / K
    diff = 10
    while diff > tol:
        for n in range(N):
            for k in range(K):
                phi_new[n, k] = beta[k, w[n]]*np.exp(digamma(gamma_old[k]) - digamma(sum(gamma_old)))
            phi_new[n, :] /= sum(phi_new[n, :])
        gamma_new = alpha + np.sum(phi_new, axis = 0)
        diff = linalg.norm(gamma_new - gamma_old) + linalg.norm(phi_new - phi_old)
        gamma_old = gamma_new
        phi_old = phi_new
    return (gamma_new, phi_new)

In [170]:
# Update beta
def up_beta(phi, B, K, V, M):
    vec = np.zeros((K, V))
    for k in range(K):
        for v in range(V):
            for m in range(M):
                vec[k, v] += sum(phi[:, k, m]*B[:, v, m])
        vec[k, :] /= sum(vec[k, :])
    return vec

In [171]:
def grad(alpha, gamma, M):
    """"""
    g = np.zeros((len(alpha)))
    for i in range(len(alpha)):
        g[i] = M*(digamma(sum(alpha)) - digamma(alpha[i])) + sum(digamma(gamma[i, :]) - digamma(sum(gamma)))
    return g

In [172]:
def Hessian(alpha, M):
    """"""
    H = np.zeros((len(alpha), len(alpha)))
    for i in range(len(alpha)):
        for j in range(len(alpha)):
            if i == j:
                H[i, j] = M*(polygamma(1, sum(alpha)) - polygamma(1, alpha[i]))
            else:
                H[i, j] = M*(polygamma(1, sum(alpha)))
    return H

In [173]:
# Update alpha
def newton_raphson(g, H, a0, gamma, M, tol = 1e-4):
    """"""
    diff = 10
    a_old = a0
    while diff > tol:
        a_new = a_old - linalg.inv(H(a_old, M)) @ g(a_old, gamma, M)
        diff = linalg.norm(a_new - a_old)
        a_old = a_new
        #print(a_new)
    return a_new

In [174]:
# M-step, update alpha and beta
def M_step(phi, B, K, V, M, g, H, a0, gamma):
    """"""
    #alpha = newton_raphson(g, H, a0, gamma, M)
    beta = up_beta(phi, B, K, V, M)
    return beta

In [175]:
# Implement E-M algorithm
alpha = alpha_true
beta = np.ones((K, V))
beta[0, :] = np.array([.2, .2, .2, .2, .2])
beta[1, :] = np.array([.1, .3, 0.1, .3, .2])
gamma = np.zeros((K, M))
phi = np.zeros((N, K, M))

it = 300 # Number of iterations of E-M algorithm
for i in range(it):
    for m in range(M):
        doc = E_step(alpha, beta, w[m,], N, K)
        gamma[:, m] = doc[0]
        phi[:, :, m] = doc[1] 
    beta = M_step(phi, B, K, V, M, grad, Hessian, np.array([1, 1]), gamma)

In [176]:
np.round(beta, 2)

array([[ 0.71,  0.23,  0.  ,  0.05,  0.  ],
       [ 0.  ,  0.04,  0.39,  0.28,  0.29]])

In [177]:
beta_true

array([[ 0.7,  0.2,  0.1,  0. ,  0. ],
       [ 0. ,  0. ,  0.3,  0.3,  0.4]])